### <center> Download XAU Data e Armazenamento</center>

--------

#### Bibliotecas

In [1]:
from minio import Minio
from pathlib import Path
from shutil import copy2
import kagglehub

#### Variáveis Locais

In [2]:
# Variáveis MinIO
MINIO_ENDPOINT = "localhost:9000"
MINIO_ACCESS_KEY = "minio"
MINIO_SECRET_KEY = "minio123"
BUCKET_NAME = "xau-lake"

In [3]:
# Variável KaggleHub
DATASET_KAGGLE = "novandraanugrah/xauusd-gold-price-historical-data-2004-2024"

#### Conectando ao MinIO e Criando/Conferindo Bucket

In [4]:
client = Minio(
    MINIO_ENDPOINT,
    access_key=MINIO_ACCESS_KEY,
    secret_key=MINIO_SECRET_KEY,
    secure=False,
)

In [5]:
if not client.bucket_exists(BUCKET_NAME):
    client.make_bucket(BUCKET_NAME)
    print(f"Bucket '{BUCKET_NAME}' criado.")
else:
    print(f"Bucket '{BUCKET_NAME}' já existe.")

Bucket 'xau-lake' já existe.


#### Baixando os Dados

In [6]:
dataset_path = kagglehub.dataset_download(DATASET_KAGGLE)

#print("Path to dataset files:", dataset_path)

In [7]:
p = Path(dataset_path)
#list(p.iterdir())

#### Copiando dados para Pasta do Projeto

In [8]:
RAW_DIR = Path("..") / "data" / "raw"
RAW_DIR.mkdir(parents=True, exist_ok=True)

In [9]:
for csv_path in p.glob("XAU_*_data.csv"):
    destino = RAW_DIR / csv_path.name
    copy2(csv_path, destino)
    print(f"Copiado: {csv_path.name} -> {destino}")

Copiado: XAU_1h_data.csv -> ../data/raw/XAU_1h_data.csv
Copiado: XAU_30m_data.csv -> ../data/raw/XAU_30m_data.csv
Copiado: XAU_1Month_data.csv -> ../data/raw/XAU_1Month_data.csv
Copiado: XAU_4h_data.csv -> ../data/raw/XAU_4h_data.csv
Copiado: XAU_1m_data.csv -> ../data/raw/XAU_1m_data.csv
Copiado: XAU_5m_data.csv -> ../data/raw/XAU_5m_data.csv
Copiado: XAU_1w_data.csv -> ../data/raw/XAU_1w_data.csv
Copiado: XAU_15m_data.csv -> ../data/raw/XAU_15m_data.csv
Copiado: XAU_1d_data.csv -> ../data/raw/XAU_1d_data.csv
